###format rules

1 строчка -- название курса

2 строчка -- пустая

3 строчка -- названия групп

4 строчка -- профили групп через ';'

5 сторочка -- начинается расписание

###imports

In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime 

import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials	

from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import math
import pandas as pd
import io

from openpyxl import load_workbook
import string

###auxiliary

In [2]:
def generate_warning_msg(text):
  return "\nWRRNING!\n"+text+"\n"

In [3]:
class Message:
  def __init__(self):
    self.__content = dict()

  def __make_tuple(self, elem):
    return elem if type(elem) == tuple else tuple(elem) if type(elem) == list else tuple([elem])

  def add_request(self, args):
    self.__content["request"] = dict(filter(lambda x: x, map(lambda x: (x[0], self.__make_tuple(x[1])) if x[1] else (), args.items())))
    
  def add_log(self, logtext):
    self.__content["log"] = logtext

  def get_content(self):
    return self.__content

  def get_json(self):
    return json.dumps(self.__content, ensure_ascii=False)


###Controller, Engine

In [35]:
class Controller:
  def __init__(self):
    self.__engine = Engine()

  def schedule_request(self, group_id = (), course = (), teacher_id = (), class_id = (), weekday_id = (), pair_numbers = ()):
    #typle args are expected
    msg = Message()
    msg.add_request({"groups": group_id,  "courses": course, "teachers": teacher_id, "classes": class_id, "weekdays": weekday_id, "pair_numbers": pair_numbers})
    return self.__engine.schedule_request(msg)

  def schedule_db_request(self):
    return self.__engine.schedule_db_request()
  


In [36]:
class Engine:
  def __init__(self):
    self.__scheduleData = DataFrame()

  def schedule_request(self, msg):
    return self.__scheduleData.request(msg)
  
  def schedule_db_request(self):
      return self.__scheduleData.get_tables()

###abstract Parser classes

In [4]:
class Parser:
  def __init__(self, res):
    self._resource = res

  def parse(self):
    pass

In [5]:
class ParserGlobal(Parser):
  def __init__(self, res):
    super().__init__(res)

class ParserLocal(Parser):
  def __init__(self, res):
    super().__init__(res)

###ParserTables

In [6]:
class ParserTables(ParserLocal):
  def parse(self, msg):
    for i in self._resource:
      if i.get_name() == "pairse":
        res = self._resource
    req = msg.get_content()["request"]

    if req["courses"]:
      req["groups"] = req.get("groups", ())+tuple((self._resource["groups"][(self._resource["groups"]["Course"].isin(req["courses"]))].index))
      req.pop("courses")

    for param in req:
      if param != "courses":
        res = res[(res[param].isin(req[param]))]    
    return res

###XlsxFile

In [166]:
class XlsxFile:
  def __init__(self, path):
    self.__LATIN_ALPHABET = ['-']+list(string.ascii_uppercase)

    self.__path = path
    self.__wb = load_workbook(self.__path)

  def unmerge_cells(self):
    print("XlsxFile: unmearging started")
    for sheet_name in self.__wb.get_sheet_names():
      sheet = self.__wb.get_sheet_by_name(sheet_name)
      self.__unmerge_cells_in_list(sheet, sheet_name)
    self.save()
    print("XlsxFile: complited")
  
  def save(self, path=''):
    self.__wb.save(path if path else self.__path) 

  def get_sheet(self, name):
    try:
      return pd.read_excel(self.__path, sheet_name=name)
    except ValueError as e:
      print(generate_warning_msg(f"no list '{name}', lists names:"), self.__wb.get_sheet_names(), "\n")

  def get_pd(self):
    sheet_list = []
    for sheet_name in self.__wb.get_sheet_names():
      sheet = self.__wb.get_sheet_by_name(sheet_name)
      sheet_list.append(pd.read_excel(self.__path, sheet_name=sheet_name))
    return sheet_list
#___________________________________________________________________________________________

  def __get_colomn_and_row(self, _range):
    c = 0
    r = ''
    for i in _range:
      if i.isalpha():
        c += self.__LATIN_ALPHABET.index(i)
      else:
        r += i
    return (c, int(r))

  def __find_not_NaN(self, startC, endC, sheet):
    for c in range(startC[0], endC[0]+1):
          for r in range(startC[1], endC[1]+1):
            if type(sheet.cell(row=r, column=c).value) == str and sheet.cell(row=r, column=c).value:
              return sheet.cell(row=r, column=c).value

  def __unmerge_cells_in_list(self, sheet, sheet_name):
    merged_Cells = list(sheet.merged_cells.ranges)    
    for _range in merged_Cells:
      startC, endC = map(self.__get_colomn_and_row, str(_range).split(':'))
      sheet.unmerge_cells(str(_range))
      v = self.__find_not_NaN(startC, endC, sheet)

      for c in range(startC[0], endC[0]+1):
        for r in range(startC[1], endC[1]+1):
          try:
            sheet.cell(row=r, column=c).value = sheet.cell(row=startC[1], column=startC[0]).value
          except Exception as e:
            print(f"unmearge warning in sheet: {sheet_name}, range: {_range}")


###ParserGoogleSheet

In [164]:
class ParserGoogleSheet(ParserGlobal):
  def __init__(self, gFile_id, files_path="ParserGoogleSheetFile.xlsx"):
    super().__init__(gFile_id)
    self.__tables = []

    self.__init_service_acc()

    self.__xlsx = self.__download_xlsx(gFile_id, files_path)
    self.__xlsx.unmerge_cells()

  def __init_service_acc(self):
    SCOPES = ['https://www.googleapis.com/auth/drive']
    SERVICE_ACCOUNT_FILE = 'central-diode-342919-c35aafd1b173.json'
    credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    pp = pprint.PrettyPrinter(indent=4)
    self.__service = build('drive', 'v3', credentials=credentials)

  def __download_xlsx(self, gFile_id, path):
    file_id = gFile_id
    request = self.__service.files().export_media(fileId=file_id,
                                                mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    filename = path
    fh = io.FileIO(filename, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print ("Download .xlsx file %d%%." % int(status.progress() * 100))

    return XlsxFile(path)

  def get_sheet(self, name):
    return self.__xlsx.get_sheet(name)

  def save(self, name):
    self.__xlsx.save(name)

  def __getitem__(self, item):
    for t in self.__tables:
      if t.get_name() == item:
        return t

  def parse(self):
    try:
      print("ParserGoogleSheet: parsing started")
      self.__tables.append(ParserTeacherTable(self).parse())
      self.__tables.append(ParserGroupTable(self).parse())
      self.__tables.append(ParserClassTable(self).parse())
      self.__tables.append(Table("weekdays", ["Weekday"], [["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота"]]))
      self.__tables.append(Table("pairs", ["Time"], [["9:00-10:25", "10:35-12:00", "12:10-13:35", "13:45-15:10", "15:20-16:45", "16:55-18:20", "18:25-19:45"]]))
      print("ParserGoogleSheet: complited")
      return self.__tables
    except Exception as e:
      print("ParserTeacherTable: WARNING!")
      print(e)


###global item parsers

In [148]:
class ParserTeacherTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS

  def __teacher_table_parser(self, sheetName):
      df =  self.__parserGS.get_sheet(sheetName)
      df["Преподаватель"] = df["Преподаватель"].apply(self.__expand_line(3, "-"))
      df["Должность"] = df["Должность"].apply(self.__expand_line(1, "-"))
      teachers = df[["Преподаватель", "Должность"]]
      return teachers

  def __expand_line(self, n, symb):
      symb = " " + symb
      return (lambda s: str(s) + symb*(n - len(str(s).split())) if s else symb*n)

  def parse(self):
    try:
      sheet_values = pd.concat([
          self.__teacher_table_parser("Список преподавателей институск"), 
          self.__teacher_table_parser("Список преподавателей баз.кафед")])
      teachers = Table("teachers", ['Surname','Name','Lastname','Post'])

      for i in range(len(sheet_values)):
        try:
          teachers.add_line(sheet_values.iloc[i]["Преподаватель"].split()+[sheet_values.iloc[i]["Должность"].lower()])
        except:
          print(sheet_values.iloc[i])
      
      teachers.sort_by("Surname")
      teachers.reset_index()
      print("ParserTeacherTable: complited")
      return teachers
    except Exception as e:
      print("ParserTeacherTable: WARNING!")
      print(e)

In [145]:
class ParserClassTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS
  
  def parse(self):
    try:
      sheet_values = self.__parserGS.get_sheet("аудиторный фонд")
      classes = list(filter(lambda x: str(x).isdigit(), sheet_values.loc[0]))
      print("ParserClassTable: complited")
      return Table("classes", ["Number"], [classes])
    except Exception as e:
      print("ParserClassTable: WARNING!")
      print(e)

In [146]:
class ParserGroupTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS

  def parse(self):
    try:
      groups = Table("groups", ['Course', 'Group'])
      for num in range(1, 7):
        sheet_values = self.__parserGS.get_sheet(str(num)+" курс ")
        group_names = list(filter(lambda x: x and not str(x).isalpha(), sheet_values.loc[1]))
        groups.add_lines([[num]*len(group_names), group_names]) #(cource, group_name)
      print("ParserGroupTable: complited")
      return groups
    except Exception as e:
      print("ParserClassTable: WARNING!")
      print(e)

In [ ]:
#was not edited after adding .xslx files support
class ParserScheduleTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS

  def __format_sheet_values(self, sheetValues):
    sheetValuesFormated = []
    prev_line = []
    for line in sheetValues[4:]:
      if line:
        line += [line[-1]]*(7-len(line))
        for cell_id in range(len(line)-1):
          if not line[cell_id]:
            line[cell_id] = prev_line[cell_id]
        prev_line = line
      sheetValuesFormated.append(line)
    return sheetValuesFormated

  def parse(self):
    schedule = Table("schedule", ['WeekdayId', 'TimeId', 'ClassId', 'GroupId', "TeacherId"])

    for num in range(1, 4):
      sheetValues = self.__parserGS.get_sheet(SheetAdress(str(num)+" курс ", "A1:P60"))
      print(self.__format_sheet_values(sheetValues))
      print()

###DataFrame, Table

In [161]:
class DataFrame:
  def __init__(self, resouce, tables = []):
    self.__tables = tables
    self.__parserGS = ParserGoogleSheet(resouce, "dataframe_file.xlsx")
    
    self.__update()

    self.__parserLocal = ParserTables(self.__tables)
    self.__lastUpdate = datetime.now()

  def __update(self):
    self.__tables = self.__parserGS.parse()

  def __check_update(self):
    update_time = 2*60*60
    if (datetime.now() - self.__lastUpdate).seconds > update_time:
      self.__update()
    return True

  def add(self, table):
    for t in self.__tables:
      if table.get_name() == t.get_name():
        raise Exception("a table with this name already exists")
    self.__tables.append(table)
  
  def get_tables(self):
    return self.__tables

  def request(self, msg):
    self.__check_update()
    return self.__parserLocal.parse(msg)
  




  def print_tables(self): #to del
    for i in self.__tables:
      print(i.get_table())
  
  

In [70]:
class Table:
  def __init__(self, name, colomns, lines=[]):
    self.__name = name
    self.__colomns = colomns
    if not lines:
      lines = [pd.Series()]*len(colomns)
    self.data = pd.DataFrame(self.__make_dir(colomns, lines))

  def __make_dir(self, keys, values):
    return {k:v for k, v in zip(keys, values)}

  def add_line(self, line):
    self.data = self.data.append(self.__make_dir(self.data.columns, line), ignore_index=True)
  
  def add_lines(self, colomns):
    for line in zip(*colomns):
      self.add_line(line)

  def get_table(self):
    return self.data

  def get_name(self):
    return self.__name

  def sort_by(self, param):
    self.data = self.data.sort_values(by=param)
  
  def reset_index(self):
    self.data = self.data.reset_index()

  def __getitem__(self, item):
    for c in self.__colomns:
      if item in list(self.data[c]):
        return  self.data[self.data[c] == item]

## tests

In [177]:
tf = DataFrame("1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc") #1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc")
tf.get_tables()

index              71
Surname         Бахнэ
Name           Сергей
Lastname            -
Post        ассистент
Name: 11, dtype: object

In [178]:
contr = Controller()
contr.schedule_request(course=1, weekday_id=[1, 2])

NameError: ignored

In [ ]:
contr.schedule_db_request()["classes"]

In [ ]:
df = pd.DataFrame({"a": pd.Series(1), "b": pd.Series(2)})
df = df.append({"a" : 3, "b": 4}, ignore_index=True)
df

## api zone

адрес сервисного аккаунта - 
service-admin-230@central-diode-342919.iam.gserviceaccount.com 	


Лимиты Google Sheets API


1.   List item
2.   List item


Google почти ко всем своим API доступ выдает БЕСПЛАТНО и без премодерации (в тестовом режиме). Однако ограничивает. К некоторым апи ограничения сильные, а к Sheets API вполне приемлемые.

    Read requests per minute 300
    Read requests per minute per user 60
    Read requests per day Unlimited

    Write requests per minute 300
    Write requests per minute per user 60
    Write requests per day Unlimited 

##downloading xls

In [155]:
cell = sheet.cell(row=13, column=1)
if type(cell).__name__ == 'MergedCell':
  print("Oh no, the cell is merged!")
else:
  print("This cell is not merged.")


This cell is not merged.


Download 100%.


In [1]:
from openpyxl import load_workbook
import string
latin_alphabet = ['-']+list(string.ascii_uppercase)
wb = load_workbook('./c4.xlsx')

In [2]:
sheet = wb.get_sheet_by_name('4 курс ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  """Entry point for launching an IPython kernel.


In [3]:
def get_colomn_and_row(_range):
  c = 0
  r = ''
  for i in _range:
    if i.isalpha():
      c += latin_alphabet.index(i)
    else:
      r += i
  return (c, int(r))

def find_not_NaN(startC, endC, sheet):
  for c in range(startC[0], endC[0]+1):
        for r in range(startC[1], endC[1]+1):
          if type(sheet.cell(row=r, column=c).value) == str and sheet.cell(row=r, column=c).value:
            return sheet.cell(row=r, column=c).value

def unmerge_cells_in_list(sheet):
  merged_Cells = list(sheet.merged_cells.ranges)
  
  for _range in merged_Cells:
    startC, endC = map(get_colomn_and_row, str(_range).split(':'))
    sheet.unmerge_cells(str(_range))
    v = find_not_NaN(startC, endC, sheet)

    for c in range(startC[0], endC[0]+1):
      for r in range(startC[1], endC[1]+1):
        sheet.cell(row=r, column=c).value = sheet.cell(row=startC[1], column=startC[0]).value

In [4]:
unmerge_cells_in_list(sheet)

In [276]:
#wb.save("c4_parsed.xlsx")

In [57]:
#pd.read_csv("c4_parsed.csv")

## gs interaction info

In [16]:
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import math
import pandas as pd
import io

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'central-diode-342919-c35aafd1b173.json'
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
pp = pprint.PrettyPrinter(indent=4)


service = build('drive', 'v3', credentials=credentials)

file_id = '1uiXDH2UXT-HaxGjuLoqHDVnOAXb4xeCMXTnSYj5rvi4' #1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc'
request = service.files().export_media(fileId=file_id,
                                             mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

filename = 'opa.xlsx'
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [47]:
#создание таблички
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'kokhanova.va@phystech.edu'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

NameError: ignored

In [ ]:
#добавление листа
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": [
    {
      "addSheet": {
        "properties": {
          "title": "Еще один лист",
          "gridProperties": {
            "rowCount": 20,
            "columnCount": 12
          }
        }
      }
    }
  ]
}).execute()


# Получаем список листов, их Id и название
spreadsheet = service.spreadsheets().get(spreadsheetId = spreadsheetId).execute()
sheetList = spreadsheet.get('sheets')
for sheet in sheetList:
    print(sheet['properties']['sheetId'], sheet['properties']['title'])
    
sheetId = sheetList[0]['properties']['sheetId']

print('Мы будем использовать лист с Id = ', sheetId)

In [46]:
#чтение
ranges = ["1 курс !A1:G51"] # 
          
results = service.spreadsheets().values().batchGet(spreadsheetId = "1uiXDH2UXT-HaxGjuLoqHDVnOAXb4xeCMXTnSYj5rvi4", #1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc", 
                                     ranges = ranges, 
                                     valueRenderOption = 'UNFORMATTED_VALUE',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
sheet_values = results['valueRanges'][0]['values']

first_pair = '9:00-10:25'

for i in range(len(sheet_values)):
  if any(map(lambda x: x == "Понедельник", sheet_values[i])):
    start_line_id = i

prev_line = []

for line in sheet_values[start_line_id:]:
  if line:
    line += [line[-1]]*(7-len(line))
    for cell_id in range(len(line)-1):
      if not line[cell_id]:
        line[cell_id] = prev_line[cell_id]
    prev_line = line
  print(line)



NameError: ignored

In [49]:
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials	

CREDENTIALS_FILE = 'central-diode-342919-c35aafd1b173.json'  # Имя файла с закрытым ключом, вы должны подставить свое

# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API 
file_id = '1uiXDH2UXT-HaxGjuLoqHDVnOAXb4xeCMXTnSYj5rvi4' #1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc'
request = service.files().export_media(fileId=file_id,
                                             mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
filename = 'Sheet.xlsx'
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

AttributeError: ignored

In [54]:
df = pd.DataFrame({'name': ['Raphael', 'Donatello'],
                   'mask': ['red', 'purple'],
                   'weapon': ['sai', 'bo staff']})

df.to_csv("t.csv")